# Bring Your Own Datatypes to TVM

In this tutorial, we will show you how you can use your own custom datatypes in TVM, utilizing TVM's Bring Your Own Datatypes framework.
Note that the Bring Your Own Datatypes framework currently only handles **software emulated versions of datatypes** right now, which is what we'll be discussing in this tutorial.

## Datatype Libraries

The central idea of the Bring Your Own Datatypes framework is to make datatypes in TVM more abstract, so that users can bring their own software-emulated datatype implementations.

In the wild, these datatype implementations usually appear as libraries. For example:
- [libposit](https://github.com/cjdelisle/libposit), a posit library
- [Stillwater Universal](https://github.com/stillwater-sc/universal), a library with posits, fixed-point numbers, and other types
- [SoftFloat](https://github.com/ucb-bar/berkeley-softfloat-3), Berkeley's software implementation of IEEE 754

The Bring Your Own Datatypes framework allows libraries such as these to be easily plugged in to TVM!

In this section, we will explore our example library, [libposit](https://github.com/cjdelisle/libposit).
**Posits** are a datatype developed to compete with IEEE 754 floating point numbers.
We won't go into much detail about the datatype itself.
If you'd like to learn more, read through John Gustafson's [Beating Floating Point at its Own Game](https://posithub.org/docs/BeatingFloatingPoint.pdf).

First, let's clone and build the library. Note that we're using my branch of libposit which includes some fixes.

In [ ]:
! git clone https://github.com/uwsampl/libposit
! cd libposit && git checkout 7c1788f291c1b5f74ded9acf4ffae7911c2df28c && autoreconf -f -i && ./configure && make

The library contains operations over posits.
Let's see what we can do with 16-bit posits (which are much like 16-bit floats):

In [ ]:
! nm libposit/libposit.a | grep posit16

We see many operations that we might expect for datatypes: `posit16_abs` for absolute value, `posit16_add` for addition, `posit16_cmp` for comparsion, etc.

Now, let's imagine we build a library which uses libposit. This is the library we want to use from TVM. Right now, we will just include a dummy function in the library, and then build it:

In [ ]:
libposit_wrapper_source = """
#include "posit.h"

void test() {
  posit16_fromf(1.0);
}
"""

! echo '{libposit_wrapper_source}' > libposit-wrapper.cc
! g++ -Ilibposit/generated -lmpfr -lgmp --std=c++14 -shared -o libposit-wrapper.so -fPIC libposit-wrapper.cc libposit/libposit.a
! ls -alh libposit-wrapper.so

*TODO: The Bring Your Own Datatypes framework is easiest to use when the datatype library is a shared library which can be loaded straight from Python with `CDLL`. libposit, on the other hand, generates a static library, and so the above code is actually a hack to turn libposit into a shared library. By writing a small wrapper library and using a libposit function, the libposit functions get linked into our shared library and can be loaded at runtime. It would be nice to have a cleaner example that doesn't require such a hack, or figure out a way around this hack.*

Let's load the library into the process space:

In [ ]:
from ctypes import CDLL, RTLD_GLOBAL

lib = CDLL('/root/libposit-wrapper.so', RTLD_GLOBAL)

Great! Now we have a library which uses libposit loaded into our process space. We will add to this library as needed and reload it.

## A Simple TVM Program
We'll begin by writing a simple program in TVM; afterwards, we will re-write it to use custom datatypes.

In [ ]:
import tvm
from tvm import relay

# Our basic program: Z = X + Y
x = relay.var('x', shape=(3, ), dtype='float32')
y = relay.var('y', shape=(3, ), dtype='float32')
z = x + y
program = relay.Function([x,y], z)

Now, we create random inputs to feed into this program using numpy:

In [ ]:
import numpy as np
np.random.seed(23) # for reproducibility

# Create random input arrays on the above context
x_input = np.random.rand(3).astype('float32')
y_input = np.random.rand(3).astype('float32')
print("x: {}".format(x_input))
print("y: {}".format(y_input))


Finally, we're ready to run the program:

In [ ]:
ex = relay.create_executor("graph")

compiled = ex.evaluate(program)

z_output = compiled(x_input, y_input)
print("z: {}".format(z_output))

## Adding Custom Datatypes

Now, we will do the same, but we will use a custom datatype for our intermediate computation.

We use the same input variables `x` and `y` as above, but before adding `x + y`, we first cast both `x` and `y` to a custom datatype via the `relay.cast(...)` call.

Note how we specify the custom datatype: we indicate it using the special `custom[...]` syntax. Additionally, note the "16" after the datatype: this is the bitwidth of the custom datatype. This tells TVM that each instance of `posit` is 16 bits wide.

In [ ]:
try:
    x_posit = relay.cast(x, dtype='custom[posit]16')
    y_posit = relay.cast(y, dtype='custom[posit]16')
    z_posit = x + y
    z = relay.cast(z_posit, dtype='float32')
except tvm.TVMError as e:
    # Print last line of error
    print(str(e).split('\n')[-1])

Trying to generate this program throws an error from TVM. Unsurprisingly, TVM does not know how to handle any custom datatype out of the box. We first have to register the custom type with TVM, giving it a name and a type code:

In [ ]:
tvm.datatype.register('posit', 150)

Note that the type code, 150, is currently chosen manually by the programmer. See `TVMTypeCode::kCustomBegin` in [include/tvm/runtime/c_runtime_api.h](https://github.com/dmlc/tvm/blob/master/include/tvm/runtime/c_runtime_api.h).

Now we can generate our program again:

In [ ]:
x_posit = relay.cast(x, dtype='custom[posit]16')
y_posit = relay.cast(y, dtype='custom[posit]16')
z_posit = x_posit + y_posit
z = relay.cast(z_posit, dtype='float32')
program = relay.Function([x, y], z)

Now that we can express our program without errors, let's try running it!

In [ ]:
try:
    compiled = ex.evaluate(program)
    z_output_posit = compiled(x_input, y_input)
    print("z: {}".format(z_output_posit))
    
except tvm.TVMError as e:
    # Print last line of error
    print(str(e).split('\n')[-1])

Now, trying to compile this program throws an error.
Let's dissect this error.

The error is occurring during the process of lowering the custom datatype code to code that TVM can compile and run. TVM is telling us that it cannot find a _lowering function_ for the `Cast` operation, when casting from source type 2 (`float`, in TVM), to destination type 150 (our custom datatype). When lowering custom datatypes, if TVM encounters an operation over a custom datatype, it looks for a user-registered *lowering function*, which tells it how to lower the operation to an operation over datatypes it understands. We have not told TVM how to lower `Cast` operations for our custom datatypes; thus, the source of this error.

To fix this error, we simply need to specify a lowering function:

In [ ]:
tvm.datatype.register_op(tvm.datatype.create_lower_func("posit16_fromf"),
                         "Cast", "llvm", "posit", "float")

The `register_op(...)` call takes a lowering function, and a number of parameters which specify exactly the operation which should be lowered with the provided lowering function. In this case, the arguments we pass specify that this lowering function is for lowering a `Cast` from `float` to `posit` for target `"llvm"`.

The lowering function passed into this call is very general: it should take an operation of the specified type (in this case, `Cast`) and return another operation which only uses datatypes which TVM understands.

In the general case, we expect users to implement operations over their custom datatypes using calls to an external library. In our example, our `posit` library implements a `Cast` from `float` to 16-bit `posit` in the function `posit16_fromf`. To provide for the general case, we have made a helper function, `create_lower_func(...)`, which does just this: given a function name, it replaces the given operation with a `Call` to the function name provided. It additionally removes usages of the custom datatype by storing the custom datatype in an opaque `uint` of the appropriate width; in our case, a `uint16_t`.

We can now re-try running the program:

In [ ]:
try:
    compiled = ex.evaluate(program)
    z_output_posit = compiled(x_input, y_input)
    print("z: {}".format(z_output_posit))
    
except tvm.TVMError as e:
    # Print last line of error
    print(str(e).split('\n')[-1])

This new error tells us that the `Add` lowering function is not found, which is good news, as it's no longer complaining about the `Cast`! We know what to do from here: we just need to register the lowering functions for the other operations in our program.

In [ ]:
tvm.datatype.register_op(tvm.datatype.create_lower_func("posit16_add"),
                         "Add", "llvm", "posit")
tvm.datatype.register_op(tvm.datatype.create_lower_func("posit16_tof"),
                         "Cast", "llvm", "float", "posit")

Now, we can run our program without errors.

In [ ]:
compiled = ex.evaluate(program)
z_output_posit = compiled(x_input, y_input)
print("z: {}".format(z_output_posit))

In [ ]:
print("x:\t\t{}".format(x_input))
print("y:\t\t{}".format(y_input))
print("z (float32):\t{}".format(z_output))
print("z (posit16):\t{}".format(z_output_posit))

Perhaps as expected, the `posit16` results are very close to the `float` results, but with some loss in precision!

## Running Models With Custom Datatypes




We will first choose the model which we would like to run with posits. In this case we use [Mobilenet](https://arxiv.org/abs/1704.04861). We choose Mobilenet due to its small size. In this alpha state of the Bring Your Own Datatypes framework, we have not implemented any software optimizations for running software emulations of custom datatypes; the result is poor performance due to many calls into our datatype emulation library.

Relay has packaged up many models within its [python/tvm/relay/testing](https://github.com/dmlc/tvm/tree/master/python/tvm/relay/testing) directory. We will go ahead and grab Mobilenet:

In [ ]:
from tvm.relay.testing.mobilenet import get_workload as get_mobilenet

module, params = get_mobilenet(image_shape=(3, 32, 32), num_classes=10)

It's easy to execute MobileNet with native TVM:

In [ ]:
ex = tvm.relay.create_executor("graph", mod=module)
input = np.random.rand(1, 3, 32, 32).astype("float32")
result = ex.evaluate()(input, **params)
print(result)

Now, we would like to change the model to use posits internally. To do so, we need to convert the network. To do this, we first define a function which will help us convert tensors:

In [ ]:
def convert_ndarray(dst_dtype, array):
    """Converts an NDArray into the specified datatype"""
    ex = relay.create_executor('graph')
    x = relay.var('x', shape=array.shape, dtype=str(array.dtype))
    cast = relay.Function([x], x.astype(dst_dtype))
    return ex.evaluate(cast)(array)


Now, to actually convert the entire network, we have written [a pass in Relay](https://github.com/gussmith23/tvm/blob/ea174c01c54a2529e19ca71e125f5884e728da6e/python/tvm/relay/frontend/change_datatype.py#L21) which simply converts all nodes within the model to use the new datatype.

In [ ]:
from tvm.relay.frontend.change_datatype import ChangeDatatype

src_dtype = 'float32'
dst_dtype = 'custom[posit]16'

# Currently, custom datatypes only work if you run simplify_inference beforehand
module = tvm.relay.transform.SimplifyInference()(module)

# Run type inference before changing datatype
module = tvm.relay.transform.InferType()(module)

# Change datatype from float to posit and re-infer types
cdtype = ChangeDatatype(src_dtype, dst_dtype)
expr = cdtype.visit(module['main'])
module = tvm.relay.transform.InferType()(tvm.relay.Module.from_expr(expr))

We also convert the parameters:

In [ ]:
params = dict(
  (p, convert_ndarray(dst_dtype, params[p])) for p in params)

We also need to convert our input:

In [ ]:
input = convert_ndarray(dst_dtype, input)

Finally, we can try to run the converted model:

In [ ]:
try:
    # Vectorization is not implemented with custom datatypes.
    with tvm.build_config(disable_vectorize=True):
        result_posit = ex.evaluate(expr)(input, **params)

except tvm.TVMError as e:
    print(str(e).split('\n')[-1])

When we attempt to run the model, we get a familiar error telling us that more funcions need to be registerd for posits. 

Because this is a neural network, many more operations are required.
Here, we register all the needed functions:

In [ ]:
tvm.datatype.register_op(
    tvm.datatype.create_lower_func("posit16_fromf"), "FloatImm",
    "llvm", "posit")
tvm.datatype.register_op(tvm.datatype.lower_ite,
                         "Call",
                         "llvm",
                         "posit",
                         intrinsic_name="tvm_if_then_else")
tvm.datatype.register_op(tvm.datatype.create_lower_func("posit16_mul"),
                         "Mul", "llvm", "posit")
tvm.datatype.register_op(tvm.datatype.create_lower_func("posit16_div"),
                         "Div", "llvm", "posit")
tvm.datatype.register_op(tvm.datatype.create_lower_func("posit16_sqrt"),
                         "Call",
                         "llvm",
                         "posit",
                         intrinsic_name="sqrt")
tvm.datatype.register_op(tvm.datatype.create_lower_func("posit16_sub"),
                         "Sub", "llvm", "posit")
tvm.datatype.register_min_func(lambda _: -268435456, "posit")
tvm.datatype.register_op(tvm.datatype.create_lower_func("posit16_exp"),
                         "Call",
                         "llvm",
                         "posit",
                         intrinsic_name="exp")

libposit_wrapper_source = """
#include "posit.h"

extern "C" posit16_t posit16_max(posit16_t a, posit16_t b) {
  return posit16_cmp(a, b) == 1 ? a : b;
}
"""

! echo '{libposit_wrapper_source}' > libposit-wrapper.cc
! g++ -Ilibposit/generated -lmpfr -lgmp --std=c++14 -shared -o libposit-wrapper-2.so -fPIC libposit-wrapper.cc libposit/libposit.a

lib = CDLL('/root/libposit-wrapper-2.so', RTLD_GLOBAL)

tvm.datatype.register_op(tvm.datatype.create_lower_func("posit16_max"),
                         "Max", "llvm", "posit")

Note that, to implement the `Max` operator, we needed to rewrite our wrapper library with a new function, `posit16_max`.
This is the only function we needed to implement by hand! All other functions we took straight from the posit library.


Now we can finally run the model:

In [ ]:
# Vectorization is not implemented with custom datatypes.
with tvm.build_config(disable_vectorize=True):
    result_posit = ex.evaluate(expr)(input, **params)
    result_posit = convert_ndarray(src_dtype, result_posit)
    print(result_posit)

Again, note that the output using 16-bit posits is understandably different from that of 32-bit floats, but is still within a sane distance:

In [ ]:
np.testing.assert_allclose(result.asnumpy(), result_posit.asnumpy(), rtol=1e-6, atol=1e-5)